In [0]:
from collections import Counter
import numpy as np
from csv import DictReader
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential, Model, load_model
from keras.layers import concatenate, Embedding, Dense, Lambda, Dropout, CuDNNLSTM, Activation, LSTM, Flatten, Input, RepeatVector, TimeDistributed, Bidirectional
from keras.optimizers import Adam, RMSprop
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, TensorBoard
import codecs
import pickle
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import keras.backend as K

Using TensorFlow backend.


In [0]:
MAX_LEN_HEAD = 100
MAX_LEN_BODY = 500
VOCAB_SIZE = 15000
EMBEDDING_DIM = 300

In [0]:
def get_vocab(lst, vocab_size):
    """
    lst: list of sentences
    """
    vocabcount = Counter(w for txt in lst for w in txt.lower().split())
    vocabcount = vocabcount.most_common(vocab_size)
    word2idx = {}
    idx2word = {}
    for i, word in enumerate(vocabcount):
        word2idx[word[0]] = i
        idx2word[i] = word[0]
    return word2idx, idx2word

In [0]:
def cov2idx_unk(lst, word2idx):
    output = []
    for sentence in lst:
        temp = []
        for word in sentence.split():
            if word in word2idx:
                temp.append(word2idx[word])
            else:
                temp.append(word2idx['<unk>'])
        temp.append(word2idx['<unk>'])
        output.append(temp)
    return output

In [0]:
def pad_seq(cov_lst, max_len=MAX_LEN_BODY):
    """
    list of list of index converted from words
    """
    pad_lst = pad_sequences(cov_lst, maxlen = max_len, padding='post')
    return pad_lst

In [0]:
label_ref = {'agree': 0, 'disagree': 1, 'discuss': 2, 'unrelated': 3}

In [0]:
def load_train_unk(file_instances, file_bodies):
    """
    article: the name of the article file
    """
    
    instance_lst = []
    # Process file
    with open(file_instances, "r", encoding='utf-8') as table:
        r = DictReader(table)
        for line in r:
            instance_lst.append(line)
            
    body_lst = []
    # Process file
    with open(file_bodies, "r", encoding='utf-8') as table:
        r = DictReader(table)
        for line in r:
            body_lst.append(line)
    
    heads = {}
    bodies = {}
    
    for instance in instance_lst:
        if instance['Headline'] not in heads:
            head_id = len(heads)
            heads[instance['Headline']] = head_id
        instance['Body ID'] = int(instance['Body ID'])
    for body in body_lst:
        bodies[int(body['Body ID'])] = body['articleBody']
            
    headData = []
    bodyData = []
    labelData = []
    for instance in instance_lst:
        headData.append(instance['Headline'])
        bodyData.append(bodies[instance['Body ID']])
        labelData.append(label_ref[instance['Stance']])
            
    
    word2idx, idx2word = get_vocab(headData+bodyData, VOCAB_SIZE)
    word2idx['<unk>'] = len(word2idx)
    
    cov_head = cov2idx_unk(headData, word2idx)
    cov_body = cov2idx_unk(bodyData, word2idx)
    remove_list = []
    for i in range(len(cov_head)):
        if len(cov_head[i])>MAX_LEN_HEAD or len(cov_body[i])>MAX_LEN_BODY:
            remove_list.append(i)
    for idx in sorted(remove_list, reverse = True):
        cov_head.pop(idx)
        cov_body.pop(idx)
        labelData.pop(idx)
    pad_head = pad_seq(cov_head, MAX_LEN_HEAD)
    pad_body = pad_seq(cov_body, MAX_LEN_BODY)
    return pad_head, pad_body, labelData, word2idx, idx2word

In [0]:
pad_head, pad_body, labelData, word2idx, idx2word = load_train_unk("train_stances.csv", "train_bodies.csv")

In [0]:
pickle.dump(word2idx, open("word2idx.pkl", "wb"))

In [0]:
#for training
train_head = pad_head[:-1000]
train_body = pad_body[:-1000]
train_label = labelData[:-1000]

val_head = pad_head[-1000:]
val_body = pad_body[-1000:]
val_label = labelData[-1000:]

In [0]:
BATCH_SIZE = 128
NUM_LAYERS = 0
HIDDEN_DIM =  512
EPOCHS = 60

In [0]:
input_head = Input(shape=(MAX_LEN_HEAD,), dtype='int32', name='input_head')
embed_head = Embedding(output_dim=EMBEDDING_DIM, input_dim=VOCAB_SIZE, input_length=MAX_LEN_HEAD)(input_head)
embed_head = Lambda(lambda x: K.reverse(x, 1))(embed_head)
# embed_head = Embedding(VOCAB_SIZE, EMBEDDING_DIM , input_length = MAX_LEN_HEAD, weights = [g_word_embedding_matrix], trainable=False)
input_body = Input(shape=(MAX_LEN_BODY,), dtype='int32', name='input_body')
embed_body = Embedding(output_dim=EMBEDDING_DIM, input_dim=VOCAB_SIZE, input_length=MAX_LEN_BODY)(input_body)
# embed_body = Embedding(VOCAB_SIZE, EMBEDDING_DIM , input_length = MAX_LEN_BODY, weights = [g_word_embedding_matrix], trainable=False)

concat = concatenate([embed_head, embed_body], axis = 1)

x = Bidirectional(CuDNNLSTM(32, return_sequences=True))(concat)
x = Bidirectional(CuDNNLSTM(32))(x)

# x = LSTM(32)(concat)
# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(4, activation='softmax', name='main_output')(x)
model = Model(inputs=[input_head, input_body], outputs=main_output)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_head (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     4500000     input_head[0][0]                 
__________________________________________________________________________________________________
input_body (InputLayer)         (None, 500)          0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 100, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
embedding_

In [12]:
wt_dir = "./"
model_path = wt_dir+'seqRbiLSTM'+'{epoch:03d}'+'.h5'
model_checkpoint = ModelCheckpoint(model_path, save_best_only =False, period =1, save_weights_only = False)
model.fit([train_head, train_body], 
          train_label, 
          epochs=2*EPOCHS, 
          validation_data=([val_head, val_body], val_label), 
          batch_size=BATCH_SIZE,
          shuffle = True, 
          callbacks=[model_checkpoint])

Train on 38357 samples, validate on 1000 samples
Epoch 1/120
38357/38357 [==============================] - 95s 2ms/step - loss: 0.8040 - acc: 0.7380 - val_loss: 0.8785 - val_acc: 0.7400
Epoch 2/120
38357/38357 [==============================] - 89s 2ms/step - loss: 0.7885 - acc: 0.7401 - val_loss: 0.7737 - val_acc: 0.7400
Epoch 3/120
38357/38357 [==============================] - 89s 2ms/step - loss: 0.7876 - acc: 0.7392 - val_loss: 0.7727 - val_acc: 0.7400
Epoch 4/120
38357/38357 [==============================] - 88s 2ms/step - loss: 0.7876 - acc: 0.7400 - val_loss: 0.7776 - val_acc: 0.7400
Epoch 5/120
38357/38357 [==============================] - 88s 2ms/step - loss: 0.7859 - acc: 0.7398 - val_loss: 0.7672 - val_acc: 0.7400
Epoch 6/120
38357/38357 [==============================] - 89s 2ms/step - loss: 0.7800 - acc: 0.7401 - val_loss: 0.7485 - val_acc: 0.7400
Epoch 7/120
38357/38357 [==============================] - 89s 2ms/step - loss: 0.7671 - acc: 0.7460 - val_loss: 0.7624 - v

KeyboardInterrupt: ignored